In [2]:
import os
import openai
import networkx as nx
import xml.etree.ElementTree as ET
import requests
from pathlib import Path
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
import chromadb
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader, UnstructuredXMLLoader
from langchain.vectorstores import Neo4jVector

In [4]:
PARENT_PATH = Path.cwd().parent
if 'publishingchatgptpocweb' not in str(PARENT_PATH):
    PARENT_PATH = PARENT_PATH / 'publishingchatgptpocweb'

CORE_DIRECTORY = PARENT_PATH / 'core'
DATA_DIRECTORY = PARENT_PATH / 'data'
MODEL_DIRECTORY = PARENT_PATH / 'models'

SPAQRQL_QUERY_FILE_PATH = CORE_DIRECTORY / 'sparql_query_template.sparql'
JATS_DATA_DIRECTORY_PATH = DATA_DIRECTORY / 'raw'
CHROMA_DB_PUB = MODEL_DIRECTORY / 'langchain_chroma_db_pub'

In [5]:
OPEN_AI_API_SECRET = os.getenv('Open__AI__API__Secret')
openai.api_key = OPEN_AI_API_SECRET

In [19]:

# This adds documents from a langchain loader to the database. The customized splitters serve to be able to break at sentence level if required.
# def add_documents(loader, instance):
#     documents = loader.load()
#     text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100, separators= ["\n\n", "\n", ".", ";", ",", " ", ""])
#     texts = text_splitter.split_documents(documents)
#     instance.add_documents(texts)


# # Create embeddings instance
embeddings = OpenAIEmbeddings(openai_api_key=OPEN_AI_API_SECRET)

# # Create Chroma instance
instance = Chroma(embedding_function=embeddings, persist_directory=str(CHROMA_DB_PUB))

# add Knowledgebase Dump (HTML file)
# for root_dir, sub_dirs, files in os.walk(JATS_DATA_DIRECTORY_PATH):
#     for file_name in files:
#         if file_name.endswith('.xml'):
#             file_path = os.path.join(root_dir, file_name)
#             loader = TextLoader(file_path)
#             documents = loader.load()
#             text_splitter = RecursiveCharacterTextSplitter(chunk_size=15000, chunk_overlap=100, separators= ["\n\n", "\n", ".", ";", ",", " ", ""])
#             # text_splitter = CharacterTextSplitter(chunk_size=15000, chunk_overlap=0)
#             texts = text_splitter.split_documents(documents)

#             # loader = UnstructuredXMLLoader(file_path, mode="single", strategy="fast")
#             # # add_documents(loader, instance)
#             # documents = loader.load()
         
#             # instance.add_documents(documents)
# print(documents)               
# print(texts)
# instance.persist()
# instance = None



# Parse the XML file
tree = ET.parse('C:/Users//MallickD//CABIGitProjects//Cabi.Publishing.ChatGPTPOC.Web//publishingchatgptpocweb//data//raw//20183000178//20183000178.xml')
root = tree.getroot()

def parse_element(element, file, indent=0):
    # Write the node name (tag) with indentation
    file.write(' ' * indent + f"Node: {element.tag}\n")
    
    # Write the node attributes, if any, with indentation
    if element.attrib:
        file.write(' ' * indent + f"Attributes: {element.attrib}\n")
    
    # Write the node text, if any, with indentation
    if element.text and element.text.strip():
        file.write(' ' * indent + f"Text: {element.text.strip()}\n")
    
    # Recursively parse the child elements with increased indentation
    for child in element:
        parse_element(child, file, indent + 2)

# Open a TXT file in write mode
with open('C:/Users//MallickD//CABIGitProjects//Cabi.Publishing.ChatGPTPOC.Web//publishingchatgptpocweb//data//raw//20183000178//20183000178.txt', 'w') as f:
    parse_element(root, f)

loader = TextLoader("C:/Users//MallickD//CABIGitProjects//Cabi.Publishing.ChatGPTPOC.Web//publishingchatgptpocweb//data//raw//20183000178//20183000178.txt")
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=15000, chunk_overlap=100, separators= ["\n\n", "\n", ".", ";", ",", " ", ""])
# text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

instance.add_documents(texts)
instance.persist()
instance = None

In [ ]:
# db = Neo4jVector.from_documents(
#     docs, embeddings, url="bolt://localhost:7687", username="neo4j", password="TGPPubCABIFirstAttempt"
# )

In [ ]:
# query = "What organisms are mentioned in article about ebola epidemic in West Africa?"
# docs_with_score = db.similarity_search_with_score(query)

# for doc, score in docs_with_score:
#     print("-" * 80)
#     print("Score: ", score)
#     print(doc.page_content)
#     print("-" * 80)

In [ ]:
# for root_dir, sub_dirs, files in os.walk(JATS_DATA_DIRECTORY_PATH):
#     for file_name in files:
#         if file_name.endswith('.xml'):
#             file_path = os.path.join(root_dir, file_name)
#             loader = UnstructuredXMLLoader(
#                 file_path,
#             )
#             docs = loader.load()
#             docs[0]

In [ ]:
# index_name = "article_composite_index"  # default index name

# store = Neo4jVector.from_existing_index(
#     embeddings,
#     url="bolt://localhost:7687", username="neo4j", password="TGPPubCABIFirstAttempt",
#     index_name=index_name,
# )

In [47]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chat_models import ChatOpenAI



instance = Chroma(persist_directory=str(CHROMA_DB_PUB), embedding_function=embeddings)

TEMPLATE = """As EVA (Expert Virtual Assistance), your role is to help user with their queries related to articles, concepts etc. Please adhere to the following guidelines:
Utilize the given context (enclosed by <ctx></ctx>) to construct your responses:
------
<ctx>
{context}
</ctx>
------

Q: {question}
A: """


# model = 'gpt-3.5-turbo-16k'
# model = 'gpt-3.5-turbo'
model = 'gpt-4'


prompt_template = PromptTemplate(
            template=TEMPLATE, 
            input_variables=["context", "question"])        

retieval_qa = RetrievalQA.from_chain_type(
            llm=ChatOpenAI(
                model_name=model, 
                openai_api_key=OPEN_AI_API_SECRET
            ),
            retriever=instance.as_retriever(search_kwargs={"k": 1}),
            chain_type_kwargs={
                "verbose": False,
                "prompt": prompt_template
            }
        )

# retieval_qa = RetrievalQAWithSourcesChain.from_chain_type(
#     ChatOpenAI(
#                 model_name="gpt-3.5-turbo", 
#                 temperature=0, 
#                 max_tokens=100, 
#                 openai_api_key=OPEN_AI_API_SECRET
#             ), chain_type="stuff", retriever=store.as_retriever(),
# )




In [50]:
import tiktoken 

encoding = tiktoken.encoding_for_model(model)

text='Can you tell me a more about the ebola epidemic in West Africa?'
text='What is the PAN numberof the article?'
text='Which preferred terms are mentioned in the article?'
text='Which article is written by M. R. Snair?'

# print('tokens:', str(len(encoding.encode(text))))
response = retieval_qa.run(text)
print(response)
# print('tokens:', str(len(encoding.encode(response))))

# retieval_qa(
#     {"question": "What is title of article?"},
#     return_only_outputs=True,
# )




The article written by M. R. Snair is "The ebola epidemic in West Africa: proceedings of a workshop."


In [31]:
def parse_element(element, file, indent=0):
    # Identify and write specific tags and attributes in a structured format
    if element.tag == "article":
        file.write("Article:\n")
    elif element.tag == "article-id":
        if 'pub-id-type' in element.attrib:
            if element.attrib['pub-id-type'] == 'CABI-pan':
                file.write(f"    PAN: {element.text.strip()}\n")
            elif element.attrib['pub-id-type'] == 'doi':
                file.write(f"    DOI: {element.text.strip()}\n")
    elif element.tag == "article-title":
        file.write(f"    Title: {element.text.strip()}\n")
    elif element.tag == "pub-date":
        file.write("    Publishing Date:\n")
        for child in element:
            if child.tag == 'day':
                file.write(f"        Day: {child.text.strip()}\n")
            elif child.tag == 'month':
                file.write(f"        Month: {child.text.strip()}\n")
            elif child.tag == 'year':
                file.write(f"        Year: {child.text.strip()}\n")

    # Recursively parse the child elements
    for child in element:
        parse_element(child, file, indent + 2)


